<a href="https://colab.research.google.com/github/ziqiandi/509-text-classification/blob/main/lgcv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn import linear_model
from sklearn.metrics import f1_score

In [ ]:
from sklearn.preprocessing import LabelEncoder 
file = '/Users/jessiding/Downloads/SIData2022.csv'
df = pd.read_csv(file)
le=LabelEncoder()
df['label']=le.fit_transform(df['label'])

In [ ]:
len(df)

5126

In [ ]:
from nltk.tokenize import sent_tokenize
from nltk import word_tokenize
import nltk
from nltk.corpus import stopwords
import string
def processing_sentence(x, stop_words):
    cut_word = nltk.sent_tokenize(x)
    words = word_tokenize(x)
    english_punctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%']
    word = [word for word in words if word not in stop_words and word not in english_punctuations and word != ' ']
    return ' '.join(word)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['content'], df['label'], test_size=0.2, stratify=df['label'])
stop_words = stopwords.words("english")
x_train = x_train.apply(lambda x: processing_sentence(x, stop_words))
x_test = x_test.apply(lambda x: processing_sentence(x, stop_words))


In [ ]:
x_train

3318    Patient brought OR wound vac take abdominal cl...
3067    Patient complained sialorrhea 8/3 per supervis...
137     Pt soiled self Changed linen gown two nurses c...
1597    Pt became agitated stated wanted go home BRT c...
4055    upon assessment pt stated chronic pressure ulc...
                              ...                        
2640    Pt admitted 5C full thickness lesions sacrococ...
535     Sedation medication titrated patient became re...
2562    As I trying orally suction patient give nebuli...
3583    While shift change I noticed blood transfusion...
4376    Expired penicillin found pharmacy freezer sinc...
Name: content, Length: 4100, dtype: object

In [ ]:
x=df['content'].apply(lambda x: processing_sentence(x, stop_words))

In [ ]:
df['content']

0       On 12/23/2021 I removed mg Hydromorphone from ...
1       Patient was supposed to be with a care compani...
2       Pt seen in burns Eval clinic. Pt sent home in ...
3       found pt's left arm swollen and warm to touch....
4       tip of syringe from surgiflo broke off and clo...
                              ...                        
5121    Admitted patient from ED via gurney, report re...
5122    Employee states he was drawing blood on a pati...
5123    Employee was walking into room 134 and slipped...
5124    At around 0910, I was called to room 122. I wa...
5125    Pt was brought to 4M 130 with PCA, from 6B. PC...
Name: content, Length: 5126, dtype: object

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=3000)
train_test = tfidf.fit_transform(x_train)
test_test = tfidf.fit_transform(x_test)

In [ ]:
x_all=tfidf.fit_transform(x)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
lrcv=LogisticRegressionCV(max_iter=10000)
lrcv.fit(train_test,y_train)

LogisticRegressionCV(max_iter=10000)

In [ ]:
train_preds = lrcv.predict(train_test)
print(len(train_preds))

4100


In [ ]:
lrcv.predict(test_test)

array([30, 12, 33, ..., 12,  3, 26])

In [ ]:
y_predict = le.inverse_transform(lrcv.predict(test_test))

In [ ]:
new_df1=pd.DataFrame()
new_df1['labels'] = y_test
new_df1['preds'] = lrcv.predict(test_test)
new_df1

,labels,preds
3254,15,30
2263,30,12
5043,7,33
1283,15,2
3754,33,7
...,...,...
2754,31,5
117,26,12
1392,3,12
4942,23,3


In [ ]:
new_df2=pd.DataFrame()
new_df2['labels'] = le.inverse_transform(y_test)
new_df2['preds'] = y_predict
new_df2

,labels,preds
0,Fall,Radiology / imaging test
1,Radiology / imaging test,Event relating to surgery or invasive procedure
2,"Complications of care (unanticipated, non surg...",Skin integrity
3,Fall,Assault (Staff/Employee)
4,Skin integrity,"Complications of care (unanticipated, non surg..."
...,...,...
1021,Respiratory care,Care coordination / communication
1022,Medication related,Event relating to surgery or invasive procedure
1023,Behavioral event,Event relating to surgery or invasive procedure
1024,Laboratory test,Behavioral event


In [ ]:
new_df3=pd.DataFrame()
new_df3['pred_total']=new_df2.groupby('preds').count()
new_df3['test_total']=new_df2.groupby('labels').count()
new_df3

,pred_total,test_total
preds,,
Adverse reaction,4,40
Assault (Staff/Employee),24,30
Behavioral event,26,65
Care coordination / communication,137,86
Complication of surgery or anesthesia,1,8
"Complications of care (unanticipated, non surgical)",141,120
Disclosure of PHI,1,9
Equipment / devices,26,12
Event relating to surgery or invasive procedure,108,39


In [ ]:
from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(estimator=lrcv,X=train_test,y=y_train,cv=6)
print("Accuracy:{:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation:{:.2f} %".format(accuracies.std()*100))

Accuracy:74.95 %
Standard Deviation:1.20 %


In [ ]:
df2 = pd.read_csv('/Users/jessiding/Downloads/SIData2022-2.csv')

In [ ]:
x_original=df2['content'].apply(lambda x: processing_sentence(x, stop_words))

In [ ]:
x_original=tfidf.fit_transform(x_original)

In [ ]:
y_original

array(['Skin integrity',
       'Complications of care (unanticipated, non surgical)',
       'Behavioral event', ..., 'Fall', 'Skin integrity',
       'Skin integrity'], dtype=object)

In [ ]:
from sklearn.preprocessing import LabelEncoder 
y_original=le.inverse_transform(lrcv.predict(x_original))

In [ ]:
le=LabelEncoder()
df2['label']=le.fit_transform(le.fit_transform(df2['label']))

In [ ]:
df2['label']

0       29
1       34
2       38
3       34
4       29
        ..
6189    15
6190    34
6191    18
6192     3
6193    29
Name: label, Length: 6194, dtype: int64

In [ ]:
x_original=df2['Description']


In [ ]:
df2['predictions'] = y_original

In [ ]:
df2

,label,content,predictions
0,Medication related,On 12/23/2021 I removed mg Hydromorphone from ...,Skin integrity
1,Other / miscellaneous,PT requested to go AMA to handle personal busi...,"Complications of care (unanticipated, non surg..."
2,Skin integrity,Patient was supposed to be with a care compani...,Behavioral event
3,Other / miscellaneous,Patient alert and oriented x4 and able to make...,Care coordination / communication
4,Medication related,Pt seen in burns Eval clinic. Pt sent home in ...,"Complications of care (unanticipated, non surg..."
...,...,...,...
6189,Exposure to blood or body fluids,Employee states he was drawing blood on a pati...,Care coordination / communication
6190,Other / miscellaneous,At 08.20 the pt tray to push2 of the exit doo...,Skin integrity
6191,Fall (Staff and Visitor),Employee was walking into room 134 and slipped...,Fall
6192,Assault (Staff/Employee),"At around 0910, I was called to room 122. I wa...",Skin integrity


In [ ]:
df2.to_csv('/Users/jessiding/Downloads/SIData2022-2的副本.csv',index=False,sep=',')